In [3]:
import torch
import numpy as np
from transformers import AutoModelForImageClassification, AutoImageProcessor
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import pipeline
from evaluate import evaluator
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [36]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(confusion_matrix):
    num_classes = len(confusion_matrix)
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracy_scores = []

    for i in range(num_classes):
        true_positives = confusion_matrix[i, i]
        false_positives = np.sum(confusion_matrix[:, i]) - true_positives
        false_negatives = np.sum(confusion_matrix[i, :]) - true_positives
        true_negatives = np.sum(confusion_matrix) - true_positives - false_positives - false_negatives

        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
        f1 = 2 * (precision * recall) / (precision + recall)
        accuracy = (true_positives) / np.sum(confusion_matrix[i, :])

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        accuracy_scores.append(accuracy)

    average_precision = np.mean(precision_scores)
    average_recall = np.mean(recall_scores)
    average_f1 = np.mean(f1_scores)
    average_accuracy  = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)

    return precision_scores, recall_scores, f1_scores, accuracy_scores, average_precision, average_recall, average_f1, average_accuracy

import numpy as np

def calculate_standard_deviation(numbers):
    if len(numbers) <= 1:
        return 0  # Standard deviation is undefined for a single value
    
    std_deviation = np.std(numbers)
    return std_deviation

In [42]:
!dir outputs_all/combine_pseudo_30%_Fold_0

all_results.json  config.json		    README.md
checkpoint-12078  eval_results.json	    trainer_state.json
checkpoint-12200  preprocessor_config.json  training_args.bin
checkpoint-854	  pytorch_model.bin	    train_results.json


In [60]:
#'outputs_all/outputs_swin/checkpoint-16900/'
#ConvNext Model vit_output
#conv_model_path = 'outputs_all/outputs_convnext/checkpoint-16900/'


#model_path='outputs_all/resnet_full_benchmark/checkpoint-16900/'
model_path = 'outputs_all/combine_pseudo_50%_Fold_0/checkpoint-13300/'
test_folder_path = 'test/original'

pipe = pipeline(
    task="image-classification",
    model=model_path, 
    device=0
)

metrics = evaluate.combine(['precision', 'recall'])


task_evaluator = evaluator("image-classification")
eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=test_folder_path,
    metric="BucketHeadP65/confusion_matrix",
    #metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    #metric=metrics.compute(["accuracy", "recall", "precision", "f1"],average='micro'),
#     confidence_level=0.95,
#     n_resamples=9999,
    label_mapping=pipe.model.config.label2id,
    #strategy="bootstrap"
)







Resolving data files:   0%|          | 0/655 [00:00<?, ?it/s]

Dataset split not defined! Automatically evaluating with split: TRAIN


Resolving data files:   0%|          | 0/655 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home/thanawit/.cache/huggingface/datasets/imagefolder/original-df25743c9d86cc29/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


In [61]:
eval_results

{'confusion_matrix': array([[ 30,  10,   8,   9],
        [  1,  70,  24,  24],
        [  1,   4, 163,   5],
        [  2,  16,  20, 268]]),
 'total_time_in_seconds': 7.208243483037222,
 'samples_per_second': 90.8681846751399,
 'latency_in_seconds': 0.011004951882499576}

In [62]:
eval_results['confusion_matrix'] 

array([[ 30,  10,   8,   9],
       [  1,  70,  24,  24],
       [  1,   4, 163,   5],
       [  2,  16,  20, 268]])

In [46]:
eval_results['confusion_matrix'] = np.array([
    [45, 4, 5, 3],
    [1, 93, 11, 14],
    [0, 5, 163, 5],
    [2, 12, 10, 282]
])

In [29]:
eval_results['confusion_matrix'] 

array([[ 45,   4,   5,   3],
       [  1,  93,  11,  14],
       [  0,   5, 163,   5],
       [  2,  12,  10, 282]])

In [41]:
eval_results

{'confusion_matrix': array([[ 38,   8,   6,   5],
        [  1,  80,  18,  20],
        [  1,   8, 160,   4],
        [  0,  10,  10, 286]]),
 'total_time_in_seconds': 8.806073516025208,
 'samples_per_second': 74.38048283472052,
 'latency_in_seconds': 0.013444387047366729}

In [68]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


precision_scores, recall_scores, f1_scores, accuracy_scores ,average_precision, average_recall, average_f1, accuracy = calculate_metrics(eval_results['confusion_matrix'])

#precision_scores, recall_scores, f1_scores, accuracy_scores, average_precision, average_recall, average_f1, average_accuracy

for i in range(len(precision_scores)):
    print(f"Metrics for class {i}:")
    print(f"Precision: {precision_scores[i]}")
    print(f"Recall: {recall_scores[i]}")
    print(f"F1 Score: {f1_scores[i]}\n")

print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1 Score: {average_f1}")
print(f"Accuracy: {accuracy}")

Metrics for class 0:
Precision: 0.8823529411764706
Recall: 0.5263157894736842
F1 Score: 0.6593406593406594

Metrics for class 1:
Precision: 0.7
Recall: 0.5882352941176471
F1 Score: 0.6392694063926941

Metrics for class 2:
Precision: 0.7581395348837209
Recall: 0.9421965317919075
F1 Score: 0.8402061855670102

Metrics for class 3:
Precision: 0.8758169934640523
Recall: 0.8758169934640523
F1 Score: 0.8758169934640523

Average Precision: 0.804077367381061
Average Recall: 0.7331411522118227
Average F1 Score: 0.753658311191104
Accuracy: 0.8106870229007633


In [39]:
pipe.model.config.label2id,

({'Atypical': '0', 'Indeterminate': '1', 'Negative': '2', 'Typical': '3'},)

In [ ]:

result = pd.DataFrame(columns=["img_name", "conv_results", "swin_results","original_label"])


In [97]:
!dir outputs_all/combine_pseudo_30%_Fold_0

all_results.json  config.json		    README.md
checkpoint-12078  eval_results.json	    trainer_state.json
checkpoint-12200  preprocessor_config.json  training_args.bin
checkpoint-854	  pytorch_model.bin	    train_results.json


In [98]:
#result = pd.DataFrame(columns=["img_name", "conv_results", "swin_results","original_label"])

total_recall = []
total_acc = []
total_f1 = []
total_prec = []
class_0_prec = []
class_0_recall = []
class_0_f1 = []
class_0_acc= []

class_2_prec= []
class_2_recall= []
class_2_f1= []
class_2_acc= []

class_3_prec= []
class_3_recall= []
class_3_f1= []
class_3_acc= []

class_1_prec= []
class_1_recall= []
class_1_f1= []
class_1_acc= []
for fold_counter in range(0,5):
    #print()
    model_path='outputs_all/combine_pseudo_30%_Fold_'+str(fold_counter)+ '/checkpoint-12200'
    #model_path='outputs_all/conv_output_50%_Fold_'+str(fold_counter)+ '/checkpoint-4300'
    test_folder_path = 'cross_val_test/30%_CV/30%_Fold_'+ str(fold_counter)+'/test/'
    print(test_folder_path)
    
    pipe = pipeline(
        task="image-classification",
        model=model_path, 
        device=0
    )

    
    metrics = evaluate.combine(['precision', 'recall'])


    task_evaluator = evaluator("image-classification")
    eval_results = task_evaluator.compute(
        model_or_pipeline=pipe,
        data=test_folder_path,
        metric="BucketHeadP65/confusion_matrix",
        #metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
        #metric=metrics.compute(["accuracy", "recall", "precision", "f1"],average='micro'),
    #     confidence_level=0.95,
    #     n_resamples=9999,
        label_mapping=pipe.model.config.label2id,
    )
    
    precision_scores, recall_scores, f1_scores, accuracy_scores, average_precision, average_recall, average_f1, average_accuracy = calculate_metrics(eval_results['confusion_matrix'])

#     for i in range(len(precision_scores)):
#         print(f"Metrics for class {i}:")
#         print(f"Precision: {precision_scores[i]}")
#         print(f"Recall: {recall_scores[i]}")
#         print(f"Recall: {accuracy_scores[i]}")
#         print(f"F1 Score: {f1_scores[i]}\n")

#     print(f"Average Precision: {average_precision}")
#     print(f"Average Recall: {average_recall}")
#     print(f"Average F1 Score: {average_f1}")
#     print(f"Accuracy: {accuracy}")
    
    class_0_prec.append(precision_scores[0])
    class_0_recall.append(recall_scores[0])
    class_0_f1.append(f1_scores[0])
    class_0_acc.append(accuracy_scores[0])
    
    class_1_prec.append(precision_scores[1])
    class_1_recall.append(recall_scores[1])
    class_1_f1.append(f1_scores[1])
    class_1_acc.append(accuracy_scores[1])
    
    class_2_prec.append(precision_scores[2])
    class_2_recall.append(recall_scores[2])
    class_2_f1.append(f1_scores[2])
    class_2_acc.append(accuracy_scores[2])
    
    class_3_prec.append(precision_scores[3])
    class_3_recall.append(recall_scores[3])
    class_3_f1.append(f1_scores[3])
    class_3_acc.append(accuracy_scores[3])


    total_acc.append(average_accuracy)
    total_f1.append(average_f1)
    total_recall.append(average_recall)
    total_prec.append(average_precision)
    

cross_val_test/30%_CV/30%_Fold_0/test/


Resolving data files:   0%|          | 0/631 [00:00<?, ?it/s]

Dataset split not defined! Automatically evaluating with split: TRAIN


Resolving data files:   0%|          | 0/631 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home/thanawit/.cache/huggingface/datasets/imagefolder/test-f213f178c42092f6/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


...

cross_val_test/30%_CV/30%_Fold_4/test/


Resolving data files:   0%|          | 0/635 [00:00<?, ?it/s]

Dataset split not defined! Automatically evaluating with split: TRAIN


Resolving data files:   0%|          | 0/635 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home/thanawit/.cache/huggingface/datasets/imagefolder/test-f304e748aea59a59/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


In [99]:
total_prec

[0.5090446371233048,
 0.49923575910378937,
 0.4817603975557257,
 0.5163317837011635,
 0.5347631675621068]

In [100]:

c0_total_acc = sum(class_0_acc) / len(class_0_acc)
c0_total_f1= sum(class_0_f1) / len(class_0_f1)
c0_total_recall=sum(class_0_recall) / len(class_0_recall)
c0_total_prec = sum(class_0_prec) / len(class_0_prec)

c1_total_acc = sum(class_1_acc) / len(class_1_acc)
c1_total_f1= sum(class_1_f1) / len(class_1_f1)
c1_total_recall=sum(class_1_recall) / len(class_1_recall)
c1_total_prec = sum(class_1_prec) / len(class_1_prec)

c2_total_acc = sum(class_2_acc) / len(class_2_acc)
c2_total_f1= sum(class_2_f1) / len(class_2_f1)
c2_total_recall=sum(class_2_recall) / len(class_2_recall)
c2_total_prec = sum(class_2_prec) / len(class_2_prec)

c3_total_acc = sum(class_3_acc) / len(class_3_acc)
c3_total_f1= sum(class_3_f1) / len(class_3_f1)
c3_total_recall=sum(class_3_recall) / len(class_3_recall)
c3_total_prec = sum(class_3_prec) / len(class_3_prec)

c0_std_acc = calculate_standard_deviation(class_0_acc)
c0_std_f1 = calculate_standard_deviation(class_0_f1)
c0_std_recall = calculate_standard_deviation(class_0_recall)
c0_std_prec = calculate_standard_deviation(class_0_prec)


c1_std_acc = calculate_standard_deviation(class_1_acc)
c1_std_f1 = calculate_standard_deviation(class_1_f1)
c1_std_recall = calculate_standard_deviation(class_1_recall)
c1_std_prec = calculate_standard_deviation(class_1_prec)


c2_std_acc = calculate_standard_deviation(class_2_acc)
c2_std_f1 = calculate_standard_deviation(class_2_f1)
c2_std_recall = calculate_standard_deviation(class_2_recall)
c2_std_prec = calculate_standard_deviation(class_2_prec)


c3_std_acc = calculate_standard_deviation(class_3_acc)
c3_std_f1 = calculate_standard_deviation(class_3_f1)
c3_std_recall = calculate_standard_deviation(class_3_recall)
c3_std_prec = calculate_standard_deviation(class_3_prec)


avg_total_acc = sum(total_acc) / len(total_acc)
avg_total_f1= sum(total_f1) / len(total_f1)
avg_total_recall=sum(total_recall) / len(total_recall)
avg_total_prec = sum(total_prec) / len(total_prec)


avg_std_acc = calculate_standard_deviation(total_acc)
avg_std_f1 = calculate_standard_deviation(total_f1)
avg_std_recall = calculate_standard_deviation(total_recall)
avg_std_prec = calculate_standard_deviation(total_prec)


In [101]:
pipe.model.config.label2id

{'Atypical': '0', 'Indeterminate': '1', 'Negative': '2', 'Typical': '3'}

In [113]:
avg_total_acc

0.642883062755032

In [79]:
print("Atypical")
print(c0_total_prec)
print(c0_std_prec)
print("-----")
print(c0_total_recall)
print(c0_std_recall)
print("-----")

Atypical
0.30503671775223495
0.08271369024499396
-----
0.16011904761904763
0.03848726254879268
-----


In [52]:
class_1_recall

[0.14545454545454545,
 0.2545454545454545,
 0.21621621621621623,
 0.17117117117117117,
 0.24324324324324326]

In [54]:
(class_0_prec[1]+class_0_prec[1]+class_0_prec[2]+class_0_prec[3]) / 4

0.28195271164021163

In [53]:
print("Interdeterminiate")
print(c1_total_prec)
print(c1_std_prec)
print("-----")
print(c1_total_recall)
print(c1_std_recall)

Interdeterminiate
0.31743794412260684
0.05783891756034666
-----
0.2061261261261261
0.04177753282036113


In [264]:
print("Negative")
print(c2_total_prec)
print(c2_std_prec)
print("-----")
print(c2_total_recall)
print(c2_std_recall)
print("-----")

Negative
0.64589932628678
0.036960976224884604
-----
0.7961132150687662
0.020145401844011035
-----


In [78]:
print("Typical")
print(c3_total_prec)
print(c3_std_prec)
print("-----")
print(c3_total_recall)
print(c3_std_recall)
print("-----")

Typical
0.7205127964953116
0.01735564573521816
-----
0.7690985603543743
0.028456066604522243
-----


In [266]:
print(avg_total_prec)
print(avg_std_prec)
print("-----")
print(avg_total_recall)
print(avg_std_recall)
print("-----")
print(avg_total_acc)
print(avg_std_acc)
print("-----")

0.49722169616423334
0.03564237764484426
-----
0.4828642372920785
0.019473547065627386
-----
0.6314845826522081
0.020350761624497386
-----


In [77]:
average_accuracy

0.662992125984252

In [113]:
pipe.model.config.label2id

{'Atypical': '0', 'Indeterminate': '1', 'Negative': '2', 'Typical': '3'}

In [114]:
 avg_std_recall

0.20614551916556126

In [115]:
avg_std_acc


0.20614551916556126

In [22]:


# Example list of numbers
data = [23, 45, 67, 89, 12, 34, 56]

std_dev = calculate_standard_deviation(data)
print("Standard Deviation:", std_dev)


Standard Deviation: 24.546498981420914


In [ ]:

model_path='outputs_all/conv_output_30%_Fold_0/checkpoint-2600'
test_folder_path = 'test/original'

pipe = pipeline(
    task="image-classification",
    model=model_path, 
    device=0
)

metrics = evaluate.combine(['precision', 'recall'])


task_evaluator = evaluator("image-classification")
eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=test_folder_path,
    metric="BucketHeadP65/confusion_matrix",
    #metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    #metric=metrics.compute(["accuracy", "recall", "precision", "f1"],average='micro'),
#     confidence_level=0.95,
#     n_resamples=9999,
    label_mapping=pipe.model.config.label2id,
    #strategy="bootstrap"
)



precision_scores, recall_scores, f1_scores, average_precision, average_recall, average_f1, accuracy = calculate_metrics(eval_results['confusion_matrix'])

for i in range(len(precision_scores)):
    print(f"Metrics for class {i}:")
    print(f"Precision: {precision_scores[i]}")
    print(f"Recall: {recall_scores[i]}")
    print(f"F1 Score: {f1_scores[i]}\n")

print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1 Score: {average_f1}")
print(f"Accuracy: {accuracy}")




In [35]:

precision_scores, recall_scores, f1_scores, accuracy_scores, average_precision, average_recall, average_f1, average_accuracy = calculate_metrics(confusion_matrix)

for i in range(len(precision_scores)):
    print(f"Metrics for class {i}:")
    print(f"Precision: {precision_scores[i]}")
    print(f"Recall: {recall_scores[i]}")
    print(f"F1 Score: {f1_scores[i]}")
    print(f"Accuracy per class: {accuracy_scores[i]}\n")

print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1 Score: {average_f1}")
print(f"Average Accuracy: {average_accuracy}")

TypeError: object of type 'function' has no len()

In [50]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics_v2(confusion_matrix):
    num_classes = len(confusion_matrix)
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracies = [] 
    for i in range(num_classes):
        true_positives = confusion_matrix[i, i]
        false_positives = np.sum(confusion_matrix[:, i]) - true_positives
        false_negatives = np.sum(confusion_matrix[i, :]) - true_positives
        total_samples = np.sum(confusion_matrix[i, :])
        
        
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
        f1 = 2 * (precision * recall) / (precision + recall)
        class_accuracy = true_positives/total_samples
        
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    average_precision = np.mean(precision_scores)
    average_recall = np.mean(recall_scores)
    average_f1 = np.mean(f1_scores)
    accuracy = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)

    return precision_scores, recall_scores, f1_scores, average_precision, average_recall, average_f1, accuracy

# Example usage
confusion_matrix = np.array([[10, 1, 2],
                             [3, 15, 1],
                             [0, 2, 12]])

precision_scores, recall_scores, f1_scores, average_precision, average_recall, average_f1, accuracy = calculate_metrics_v2(confusion_matrix)

for i in range(len(precision_scores)):
    print(f"Metrics for class {i}:")
    print(f"Precision: {precision_scores[i]}")
    print(f"Recall: {recall_scores[i]}")
    print(f"F1 Score: {f1_scores[i]}\n")

print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1 Score: {average_f1}")
print(f"Accuracy: {accuracy}")

Metrics for class 0:
Precision: 0.7692307692307693
Recall: 0.7692307692307693
F1 Score: 0.7692307692307693

Metrics for class 1:
Precision: 0.8333333333333334
Recall: 0.7894736842105263
F1 Score: 0.8108108108108109

Metrics for class 2:
Precision: 0.8
Recall: 0.8571428571428571
F1 Score: 0.8275862068965518

Average Precision: 0.800854700854701
Average Recall: 0.8052824368613843
Average F1 Score: 0.802542595646044
Accuracy: 0.8043478260869565


In [64]:
precision_scores, recall_scores, f1_scores, accuracy_scores ,average_precision, average_recall, average_f1, accuracy = calculate_metrics(confusion_matrix)


In [198]:
import numpy as np

def class_accuracy_recall_precision(confusion_matrix):
    num_classes = confusion_matrix.shape[0]
    accuracies = np.zeros(num_classes)
    recalls = np.zeros(num_classes)
    precisions = np.zeros(num_classes)
    
    for i in range(num_classes):
        true_positives = confusion_matrix[i, i]
        total_samples = np.sum(confusion_matrix[i, :])
        actual_positives = np.sum(confusion_matrix[:, i])
        
        if total_samples > 0:
            accuracies[i] = true_positives / total_samples
        
        if actual_positives > 0:
            recalls[i] = true_positives / actual_positives
        
        if true_positives + total_samples - actual_positives > 0:
            precisions[i] = true_positives / (true_positives + total_samples - actual_positives)
    
    return accuracies, recalls, precisions

# Example confusion matrix (replace this with your own confusion matrix)
confusion_matrix = np.array([[90, 5, 20],
                             [3, 85, 12],
                             [8, 10, 82]])

class_accuracies, class_recalls, class_precisions = class_accuracy_recall_precision(confusion_matrix)

for i, (accuracy, recall, precision) in enumerate(zip(class_accuracies, class_recalls, class_precisions)):
    print(f"Class {i}: Accuracy={accuracy:.2f}, Recall={recall:.2f}, Precision={precision:.2f}")


Class 0: Accuracy=0.78, Recall=0.89, Precision=0.87
Class 1: Accuracy=0.85, Recall=0.85, Precision=1.00
Class 2: Accuracy=0.82, Recall=0.72, Precision=1.21


In [199]:
precision_scores, recall_scores, f1_scores, accuracy_scores ,average_precision, average_recall, average_f1, accuracy = calculate_metrics(confusion_matrix)


In [202]:
accuracy

0.8158730158730159

In [201]:
precision_scores

[0.8910891089108911, 0.85, 0.7192982456140351]

In [200]:
recall_scores

[0.782608695652174, 0.85, 0.82]

In [191]:
import numpy as np

def calculate_precision(confusion_matrix):
    num_classes = len(confusion_matrix)
    precision = np.zeros(num_classes)
    
    for i in range(num_classes):
        true_positive = confusion_matrix[i, i]
        false_positive = np.sum(confusion_matrix[:, i]) - true_positive
        
        if true_positive + false_positive == 0:
            precision[i] = 0.0
        else:
            precision[i] = true_positive / (true_positive + false_positive)
    
    return precision

# # Example confusion matrix for 3 classes: [[C00, C01, C02], [C10, C11, C12], [C20, C21, C22]]
# confusion_matrix = np.array([[50, 5, 5],
#                              [10, 80, 10],
#                              [5, 5, 50]])

precision = calculate_precision(confusion_matrix)
print("Precision for each class:", precision)


Precision for each class: [0.89108911 0.85       0.82828283]


In [166]:
import numpy as np

def calculate_precision_recall(confusion_matrix):
    num_classes = len(confusion_matrix)
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)
    
    for i in range(num_classes):
        true_positive = confusion_matrix[i, i]
        false_positive = np.sum(confusion_matrix[:, i]) - true_positive
        false_negative = np.sum(confusion_matrix[i, :]) - true_positive
        
        if true_positive + false_positive == 0:
            precision[i] = 0.0
        else:
            precision[i] = true_positive / (true_positive + false_positive)
        
        if true_positive + false_negative == 0:
            recall[i] = 0.0
        else:
            recall[i] = true_positive / (true_positive + false_negative)
    
    return precision, recall

# # Example confusion matrix for 3 classes: [[C00, C01, C02], [C10, C11, C12], [C20, C21, C22]]
# confusion_matrix = np.array([[50, 5, 5],
#                              [10, 80, 10],
#                              [5, 10, 85]])

precision, recall = calculate_precision_recall(confusion_matrix)
print("Precision for each class:", precision)
print("Recall for each class:", recall)


Precision for each class: [0.89108911 0.85       0.82828283]
Recall for each class: [0.9  0.85 0.82]
